In [ ]:
from PIL import Image
import numpy as np
import scipy
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import sys
import os
from multiprocessing import Pool
from functools import partial
directory = 'figures'
if not os.path.exists(directory):
    os.makedirs(directory)
from concurrent.futures import ProcessPoolExecutor, as_completed
from dask.distributed import Client, progress
from dask import compute, delayed
import dask.array as da
from dask.diagnostics import ProgressBar
plt.rcParams['figure.dpi']=400

# Problem 5

In [ ]:
def mps(psi, k):
    
    mps_tensors = []
    L = int(np.log2(len(psi)))  # Number of spins/sites
    psi_matrix = psi.reshape(2, 2**(L-1))

    for i in range(L-1):
        
        U, S, Vh = np.linalg.svd(psi_matrix, full_matrices=False)
        
        chi = min(k, len(S))
        
        S_truncated = S[: chi]
        U_truncated = U[:, :chi]
        Vh_truncated = Vh[:chi, :]
        
        if i == 0:
            mps_tensors.append(U_truncated) 
            
        else:
            mps_tensor.append(U_truncated.reshape((-1, 2, chi)))
            mps_tensors.append(mps_tensor)

        if i < L - 2:
            psi_matrix = (np.diag(S_truncated) @ Vh_truncated).reshape(2*chi, -1)
            
        else:
            last_tensor = (np.diag(S_truncated) @ Vh_truncated)
            mps_tensors.append(last_tensor)

    return mps_tensors


In [ ]:
def sparseH(L, J, h, periodic):
    
    """
    generates the sparse Hamiltonian matrix for the quantum Ising chain
    
        Parameters:
            L (int): length of chain
            J (float): ising interaction strength
            h (float): magnetic field strength
            periodic (bool): does the chain have periodic boundary conditions?
            
        Returns:
            H (csr_matrix): sparse matrix representing the Hamiltonian operator
    """
    
    dim = 2 ** L # dimensions of the Hilbert space
    
    # initialize 
    H_data = []
    H_rows = []
    H_cols = []
    
    "Calculation of off-diagonal elements due to the magnetic field"
    
    for beta in range(dim): # iterate over all states
        
        for j in range(1, L + 1): # iterate over all sites
            
            alpha = beta ^ (1 << (j - 1)) # flips jth bit of beta to get the state alpha that is related to beta by a single bit flip
            
            "Keep track of the indices with non-zero matrix elements"
            
            H_data.append(-h)
            H_rows.append(alpha)
            H_cols.append(beta)
    
    "Calculation of diagonal elements due to Ising interaction"

    for alpha in range(dim):  # iterate over all states
        
        A = 0
        
        for j in range(1, L): # iterate over all sites
            
            if 2 * (alpha & (1 << (j - 1))) == alpha & (1 << j): # check if site j and j+1 have the same spin
                
                A -= J # if they do, decrease the energy by the ising interaction term
                
            else:
                
                A += J # if not, increase the energy by the ising interaction term
                
        "Handling periodic boundary conditions"
                
        if periodic and L > 1: # L > 1 needed for periodicity to mean anything
            
            if (alpha & (1 << (L - 1))) == ((alpha & (1 << 0)) * (2 ** (L - 1))): # Check if the states at either end have the same spin
                
                A -= J # if they do, decrease the energy by the ising interaction term
                
            else:
                
                A += J # if not, increase the energy by the ising interaction term
        
        if A != 0: # Check if the resulting matrix element is non-zero, if so, keep track of it
        
            H_data.append(A)
            H_rows.append(alpha)
            H_cols.append(alpha)

    H_data = np.array(H_data, dtype=float) # convert the list into a np array
    
    H = scipy.sparse.csr_matrix((H_data, (H_rows, H_cols)), shape=(dim, dim), dtype=np.float64) # make it into a csr sparse matrix
    
    return H

def psi_gs(L, J, h, periodic):
    H = sparseH(L, J, h, periodic)
    return scipy.sparse.linalg.eigsh(H, k=1, which='SA')[1]

In [ ]:
def contract_mps_tensors(mps_tensors):
    state = mps_tensors[0]

    for i, tensor in enumerate(mps_tensors[1:], 1):
        if i == len(mps_tensors) - 1:
            state = np.einsum('...j,jk->...k', state, tensor)
        else:
            state = np.einsum('...j,jkl->...kl', state, tensor)

    return state.flatten()

In [ ]:
L = 10
J = 1
h = 1.25
periodic = False

psi_ground = psi_gs(L, J, h, periodic)

k_values = range(2, 21)
overlaps = []

def overlap(state1, state2):
    return np.conj(state1).T @ state2

for k in k_values:
    mps_tensors = mps(psi_ground, k)
    psi_approx = contract_mps_tensors(mps_tensors)
    overlaps.append(overlap(psi_ground, psi_approx))

plt.plot(k_values, overlaps, marker='o')
plt.xlabel('Bond Dimension (k)')
plt.ylabel('Overlap ⟨ψ̃gs(k)|ψgs⟩')
plt.title('Overlap vs. Bond Dimension for Ground State Approximation')
plt.savefig(os.path.join(directory, 'overlapbond.png'), dpi = 400)
plt.grid(True)
plt.show()

In [ ]:
L = 15
J = 1
h = 1.25
periodic = False

psi = psi_gs(L, J, h, periodic)

K = range(1, 30)
storage = []         

for k in K:
    mps_psi = mps(psi, k)
    total_storage = 0
    for tensor in mps_psi:
        total_storage += np.prod(tensor.shape)  # Product of the dimensions of the tensor
    
    storage.append(total_storage)

storage = (np.array(storage))/2**L

plt.figure(figsize=(10, 5))
plt.plot(K, storage, marker='o')
plt.xlabel('Bond Dimension $k$')
plt.ylabel('Storage (ratio w.r.t. saving as a coefficient tensor)')
plt.title('Storage Requirements for Different Bond Dimensions $k$')
plt.grid(True)
plt.gca().invert_xaxis()
plt.savefig(os.path.join(directory, 'storagesavings.png'), dpi = 400)
plt.show()


In [ ]:
def inner_product(bra, ket):
    result = np.einsum('ij,im->jm', bra[0].conj(), ket[0])
    
    for i in range(1, len(ket) - 1):

        A = np.einsum('jkl,mkn->jmln', bra[i].conj(), ket[i])
        result = np.einsum('jm,jmln->ln', result, A)
    
    final_result = np.einsum('jm,jm', result, np.einsum('ji,mi->jm', bra[-1].conj(), ket[-1]))

    return final_result

psi = psi_gs(10, np.pi, 0.4, False).reshape(-1)
mps_psi = mps(psi, 2**1)
print(inner_product(mps_psi, mps_psi))


In [ ]:
def magnetic_term(ket):
    
    X = np.array([[0, 1], [1, 0]])
    
    L = len(ket)
    results = np.zeros(L)
    
    for i in range(L):
        
        Hket = [np.copy(k) for k in ket]
        
        if i == 0:
            Hket[i] = np.einsum('ij,ik->jk', X, ket[i])

        else:
            Hket[i] = np.einsum('ij,ki...->kj...', X, ket[i])
        
        results[i] = inner_product(ket, Hket)
        
    return np.sum(results)
        

def ising_term(ket):
    
    Z = np.array([[1,0],[0,-1]])
    L = len(ket)
    results = np.zeros(L)
    
    for i in range(0, L-1):
        
        Hket = [np.copy(k) for k in ket]
        
        if i == 0:
            Hket[i] = np.einsum('ij,ik->jk', Z, ket[i])            
        else:
            Hket[i] = np.einsum('ij,ki...->kj...', Z, ket[i])
        
        Hket[i+1] = np.einsum('ij,ki...->kj...', Z, ket[i+1])
        
        results[i] = inner_product(ket, Hket)
        
    return np.sum(results)

def expectation_value_energy(L, J, h, periodic, k):
    
    ket = mps(psi_gs(L, J, h, periodic), k)
    
    energy = -h*magnetic_term(ket)-J*ising_term(ket)
    norm = inner_product(ket, ket)
    
    return energy/norm

In [ ]:
    
print(expectation_value_energy(10, 1, 0.5, False, 3))
     
H=sparseH(10, 1, 0.5, False)
print(scipy.sparse.linalg.eigsh(H, k=1, which='SA')[0])            
            
psi = psi_gs(10, 1, 0.5, False).reshape(-1)
mps_psi = mps(psi, 2**10)
norm = inner_product(mps_psi, mps_psi)
print((-1*ising_term(mps_psi)-0.5*magnetic_term(mps_psi))/norm)           
        
    

In [ ]:
ks = range(2,30)
h = 1.25
L = 15

energies=[]
H = sparseH(L, 1, h, False)
Egs = scipy.sparse.linalg.eigsh(H, k=1, which='SA')[0]

for k in ks:
    energies.append(expectation_value_energy(L, 1, h, False, k))


In [ ]:
plt.plot(ks, energies, label = 'convergence')
plt.axhline(y=Egs, color='r', linestyle='--', label='Exact Ground State Energy')
plt.legend()
plt.title(f'Convergence of the expectation value of energy vs bond-dimension L = {L}')
plt.ylabel('Energy (k)')
plt.xlabel('Bond dimension (k)')
plt.savefig(os.path.join(directory, 'energyconvergencenearcrit.png'), dpi = 400)
plt.show()

In [ ]:
def expectation_value_Czz(L, J, h, periodic, k, r):
    
    Z = np.array([[1,0],[0,-1]])
    ket = mps(psi_gs(L, J, h, periodic), k)
    
    Czzket = [np.copy(k) for k in ket]
    
    Czzket[0] = np.einsum('ij,ik->jk', Z, ket[0])
    Czzket[r] =np.einsum('ij,ki...->kj...', Z, ket[r])
    
    return inner_product(ket, Czzket)

In [ ]:
def Czz(vec, r):
    """
    generates the correlation function Czz(r) between sites 1 and 1+r
    
        Parameters:
            vec (np.ndarray): eigenvector
            r (integer): site
            
        Returns:
            C (float): correlation of sigma_z between site 1 and 1+r
    """
    
    dim = len(vec) 
    Cvec = np.zeros((dim, 1)) #initialize
    
    for alpha in range(dim): # iterate over basis
        
        if (alpha & (1 << (r))) == (2**(r))*(alpha & (1 << 0)):
            
            Cvec[alpha] = vec[alpha] # if site 1 and 1+r have spin, keep the coefficient unchanged
            
        else:
            Cvec[alpha] = -vec[alpha] # else multiply it by -1
    
    C = np.dot(np.conj(vec).T, Cvec)[0][0] # dot product of vector with the vector output from the operator action
    
    return C

In [ ]:
L = 15
J = 1
h = 1
r = 5
vec = psi_gs(L, J, h, periodic)
Czz_actual = Czz(vec, 5)
ks = range(2,30)
observable = []
for k in ks:
    observable.append(expectation_value_Czz(L, 1, h, False, k, r))


In [ ]:
plt.plot(ks, observable, label = 'convergence')
plt.axhline(y=Czz_actual, color='r', linestyle='--', label='Exact value')
plt.legend()
plt.title(f'Convergence of the expectation value of Czz vs bond-dimension L = {L}')
plt.ylabel('Czz (k)')
plt.xlabel('Bond dimension (k)')
plt.savefig(os.path.join(directory, 'Czzconvergencecrit.png'), dpi = 400)
plt.show()
    

In [ ]:
def expectation_value_Cxx(L, J, h, periodic, k, r):
    
    X = np.array([[0,1],[1,0]])
    ket = mps(psi_gs(L, J, h, periodic), k)
    
    Cxxket = [np.copy(k) for k in ket]
    
    Cxxket[0] = np.einsum('ij,ik->jk', X, ket[0])
    Cxxket[r] = np.einsum('ij,ki...->kj...', X, ket[r])
    
    return inner_product(ket, Cxxket)

In [ ]:
def Cxx(vec, r):
    
    """
    generates the correlation function Cxx(r) between sites 1 and 1+r
    
        Parameters:
            vec (np.ndarray): eigenvector
            r (integer): site
            
        Returns:
            C (float): correlation of sigma_x between site 1 and 1+r
    """
    
    dim = len(vec)
    Cvec = np.zeros((dim, 1)) # initialize
    
    for beta in range(dim): # iterate over basis
        
        alpha = (beta ^ (1<<0)) ^ (1<<r) # flip 1 and 1+r site to map the state to that which is connected by two bit flips
        
        Cvec[alpha] += vec[beta] # record this state
        
    C = np.dot(np.conj(vec).T, Cvec)
    
    return C[0][0]

In [ ]:
L = 15
J = 1
h = 1.25
r = 5
periodic = False
vec = psi_gs(L, J, h, periodic)
Cxx_actual = Cxx(vec, r)
ks = range(2,30)
observable = []
for k in ks:
    observable.append(expectation_value_Cxx(L, 1, h, False, k, r))
plt.plot(ks, observable, label = 'convergence')
plt.axhline(y=Cxx_actual, color='r', linestyle='--', label='Exact Value')
plt.legend()
plt.title(f'Convergence of the expectation value Cxx vs bond-dimension L = {L}')
plt.ylabel('Cxx (k)')
plt.xlabel('Bond dimension (k)')
plt.savefig(os.path.join(directory, 'Cxxconvergencenearcrit.png'), dpi = 400)
plt.show()
    

# Problem 5.2

In [ ]:
def compute_vector_optimized(state):
    binary_state = [int(x) for x in format(state, '0{}b'.format(L))]
    result_matrix = A1[binary_state[0]]
    for i in range(1, L):
        if i != L - 1:
            result_matrix = result_matrix @ Aell[binary_state[i]]
        else:
            result_matrix = result_matrix @ AL[binary_state[i]]
    return result_matrix[0]
def analyze_mps_schmidt_values(wavefunction, L):
    schmidt_values = {}
    for l in range(L-1, 0, -1):
        psi_matrix = wavefunction.reshape((2**l, 2**(L-l)))
        U, S, Vh = np.linalg.svd(psi_matrix, compute_uv=True)
        schmidt_values[l] = S[:2]  # Assuming bond dimension 2
    return schmidt_values


In [ ]:
A1 = [np.array([1, 1]) / np.sqrt(2), np.array([1, -1]) / np.sqrt(2)]
Aell = [ np.array([[1, 1], [0, 0]])/ np.sqrt(2), np.array([[0, 0], [1, -1]])/ np.sqrt(2)]
AL = [np.array([[1], [0]]), np.array([[0], [1]])]

L = 10
wavefunction = np.zeros(2**L)
for i in range(2**L):
    wavefunction[i] = compute_vector_optimized(i)

# wavefunction = wavefunction / np.linalg.norm(wavefunction)

schmidt_values = analyze_mps_schmidt_values(wavefunction, L)
for cut, values in schmidt_values.items():
    print(f"Cut at position {cut}: Schmidt values = {values}")

In [ ]:
A1 = [np.array([1, 2]), np.array([2, -1])]
Aell = [ np.array([[1, 1], [0, 0]])/np.sqrt(2), np.array([[0, 0], [1, -1]])/np.sqrt(2)]
AL = [np.array([[1], [3]]), np.array([[3], [1]])]

L = 10
wavefunction = np.zeros(2**L)
for i in range(2**L):
    wavefunction[i] = compute_vector_optimized(i)

# wavefunction = wavefunction / np.linalg.norm(wavefunction)

schmidt_values = analyze_mps_schmidt_values(wavefunction, L)
for cut, values in schmidt_values.items():
    print(f"Cut at position {cut}: Schmidt values = {values}")

Converting them into tensors

In [ ]:
def canonical_converter(mps_tensors, k, center):
    new_mps_tensors = mps_tensors.copy()
    L = len(new_mps_tensors)
    for i in range(L-1):
        
        W = np.einsum('ijk,klm->ijlm', new_mps_tensors[i], new_mps_tensors[i+1])
        
        Wmatrix = W.reshape((W.shape[0]*2, -1))
        
        U, sv, Vt = np.linalg.svd(Wmatrix, full_matrices=False)
        chi = min(k, len(sv))
        
        U = U[:, :chi]
        new_mps_tensors[i] = U.reshape(-1, 2, chi)
        
        S = np.diag(sv[:chi])
        Vt = Vt[:chi, :]
        

        new_mps_tensors[i+1] = (S @ Vt).reshape(chi, 2, -1)
    
    schmidt_tracker=[]

        
    for i in range(L-1, 0, -1):
        
                
        W = np.einsum('ijk,klm->ijlm', new_mps_tensors[i-1], new_mps_tensors[i])

        Wmatrix = W.reshape((W.shape[0]*2, -1))
        
        U, sv, Vt = np.linalg.svd(Wmatrix, full_matrices=False)
        chi = min(k, len(sv))
        
        U = U[:, :chi]
        S = np.diag(sv[:chi])
        Vt = Vt[:chi, :]
        
        new_mps_tensors[i] = Vt.reshape(chi, 2, -1)
        
        schmidt_tracker.append(sv[:chi])
        
        new_mps_tensors[i-1] = (U@S).reshape(-1, 2, chi)
            
        if i == center:
            break

    return np.array(schmidt_tracker), new_mps_tensors, S

In [ ]:
A1 = np.stack((np.array([1, 2]), np.array([2, -1])), axis = 0)
A1 = np.expand_dims(A1, axis=0)
Aell = np.stack(((np.array([[1, 1], [0, 0]])/np.sqrt(2)), (np.array([[0, 0], [1, -1]])/np.sqrt(2))), axis = 1)
AL = np.stack((np.array([[1], [3]]), np.array([[3], [1]])), axis = 1)

mps_tensors=[]

for i in range(10):
    if i == 0:
        mps_tensors.append(A1)
    elif i == L - 1:
        mps_tensors.append(AL)
    else:
        mps_tensors.append(Aell)

print(canonical_converter(mps_tensors, 2, 0)[0])


The Schmidt values match!